In [1]:
import math
from scipy.sparse import dok_matrix
import numpy as np

In [2]:
def load_input():
    with open("input") as infile:
        data = infile.read()
    lines = data.split("\n")
    first_wire = lines[0].split(",")
    second_wire = lines[1].split(",")
    return first_wire, second_wire

In [3]:
def manhattan(point):
    return abs(point[0]) + abs(point[1])

In [4]:
class Wire:
    def __init__(self, instructions, capacity_x=40000, capacity_y=40000):
        self.panel = dok_matrix((capacity_x, capacity_y), dtype=np.byte)
        
        self.central_port = (math.floor(capacity_x/2), math.floor(capacity_y/2))
        self.reset_pos()
        self.draw("o")
        
        self.instructions = instructions
        self.walk()

    def real_panel_dimensions(self):
        topleft = None
        bottomright = None

        rows, cols = self.panel.nonzero()
        for x in rows:
            for y in cols:
                if topleft:
                    if x < topleft[0]:
                        topleft[0] = x
                    if y < topleft[1]:
                        topleft[1] = y
                else:
                    topleft = [x, y]
                    
                if bottomright:
                    if x > bottomright[0]:
                        bottomright[0] = x
                    if y > bottomright[1]:
                        bottomright[1] = y
                else:
                    bottomright = [x, y]
                        
        return topleft, bottomright
        
    def __repr__(self):
        topleft, bottomright = self.real_panel_dimensions()
        
        width = bottomright[0] - topleft[0] + 1 + 2   # 1 to include extremity, 2 for 1-margin each side
        height = bottomright[1] - topleft[1] + 1 + 2
        
        s = ""
        for y in range(height):
            for x in range(width):
                c = self.panel[topleft[0] + x - 1, topleft[1] + y - 1]
                if c == 0:
                    s += "."
                else:
                    s += chr(c)
            s += "\n"
        return s
        
    def draw(self, char):
        self.panel[self.x, self.y] = ord(char)

    def reset_pos(self):
        self.x = self.central_port[0]
        self.y = self.central_port[1]

    def walk(self, upto=None):
        self.reset_pos()
        length = 0

        for i, instruction in enumerate(self.instructions):
            direction = instruction[0]
            distance = int(instruction[1:])
            if direction == "R":
                for x in range(distance):
                    self.x += 1
                    length += 1
                    if x == distance-1 and i != len(self.instructions)-1:
                        self.draw("+")
                    else:
                        self.draw("-")
                    
                    if upto == (self.x - self.central_port[0], self.central_port[1] - self.y):
                        return length
            elif direction == "L":
                for x in range(distance):
                    self.x -= 1
                    length += 1
                    if x == distance-1 and i != len(self.instructions)-1:
                        self.draw("+")
                    else:
                        self.draw("-")

                    if upto == (self.x - self.central_port[0], self.central_port[1] - self.y):
                        return length
            elif direction == "U":
                for y in range(distance):
                    self.y -= 1
                    length += 1
                    if y == distance-1 and i != len(self.instructions)-1:
                        self.draw("+")
                    else:
                        self.draw("|")

                    if upto == (self.x - self.central_port[0], self.central_port[1] - self.y):
                        return length
            elif direction == "D":
                for y in range(distance):
                    self.y += 1
                    length += 1
                    if y == distance-1 and i != len(self.instructions)-1:
                        self.draw("+")
                    else:
                        self.draw("|")

                    if upto == (self.x - self.central_port[0], self.central_port[1] - self.y):
                        return length
        
        return length

In [5]:
w1 = Wire(["R8","U5","L5","D3"])

In [6]:
w1.real_panel_dimensions()

([20000, 19995], [20008, 20000])

In [7]:
w1.central_port

(20000, 20000)

In [8]:
w1

...........
....+----+.
....|....|.
....|....|.
....|....|.
.........|.
.o-------+.
...........

In [9]:
w2 = Wire(["U7","R6","D4","L4"])

In [10]:
w2

.........
.+-----+.
.|.....|.
.|.....|.
.|.....|.
.|.----+.
.|.......
.|.......
.o.......
.........

In [11]:
class Panel(Wire):
    def __init__(self, w1, w2):
        assert(w1.panel.shape == w2.panel.shape)
        assert(w1.central_port == w2.central_port)

        self.w1 = w1
        self.w2 = w2
        self.panel = w1.panel.tocsc() + w2.panel.tocsc() # efficient matrix operation CSC + CSC

        self.central_port = w1.central_port
        self.reset_pos()
        self.draw("o")
        
        self.wire_intersections = []
        self.compute_intersection()

    def compute_intersection(self):
        rows, cols = self.panel.nonzero()
        nonzeros = zip(rows, cols)
        skipchars = [0, ord('+'), ord('-'), ord('|'), ord('o')]
        for (x, y) in nonzeros:
            c = self.panel[x, y]
            if c not in skipchars:
                self.panel[x, y] = ord('X')
                intersection = (x - self.central_port[0], self.central_port[1] - y)
                if intersection not in self.wire_intersections:
                    self.wire_intersections.append(intersection)

    def closest_wire_cross(self):
        closest = None
        for intersection in self.wire_intersections:
            mdist = manhattan(intersection)
            if closest:
                if mdist < manhattan(closest):
                    closest = intersection
            else:
                closest = intersection
        return closest

    def closest_distance(self):
        closest = self.closest_wire_cross()
        return manhattan(closest)
    
    def shortest_walking_length(self):
        shortest = None
        shortest_length = None
        for intersection in self.wire_intersections:
            length1 = self.w1.walk(upto=intersection)
            length2 = self.w2.walk(upto=intersection)
            length = length1 + length2
            if shortest:
                if length < shortest_length:
                    shortest = intersection
                    shortest_length = length
            else:
                shortest = intersection
                shortest_length = length

        return shortest_length

In [12]:
p = Panel(w1, w2)

In [13]:
p

...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........

In [14]:
p.wire_intersections

[(3, 3), (6, 5)]

In [15]:
p.closest_wire_cross()

(3, 3)

In [16]:
p.closest_distance()

6

In [17]:
w1.walk(upto=(3, 3))

20

In [18]:
w2.walk(upto=(3, 3))

20

In [19]:
w1.walk(upto=(6, 5))

15

In [20]:
w2.walk(upto=(6, 5))

15

In [21]:
p.shortest_walking_length()

30

In [22]:
w1 = Wire(["R75","D30","R83","U83","L12","D49","R71","U7","L72"])

In [23]:
w2 = Wire(["U62","R66","U55","R34","D71","R55","D58","R83"])

In [24]:
p = Panel(w1, w2)

In [25]:
p.wire_intersections

[(146, 46), (155, 11), (155, 4), (158, -12)]

In [26]:
p.closest_distance()

159

In [27]:
p.shortest_walking_length()

610

In [28]:
w1 = Wire(["R98","U47","R26","D63","R33","U87","L62","D20","R33","U53","R51"])

In [29]:
w2 = Wire(["U98","R91","D20","R16","D67","R40","U7","R15","U6","R7"])

In [30]:
p = Panel(w1, w2)

In [31]:
p.closest_distance()

135

In [32]:
p.shortest_walking_length()

410

In [33]:
wire1, wire2 = load_input()

In [34]:
w1 = Wire(wire1)

In [35]:
w2 = Wire(wire2)

In [36]:
p = Panel(w1, w2)

In [37]:
p.closest_distance()

1337

In [38]:
p.shortest_walking_length()

65356